<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Asi_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through NLTK, a Python NLP package.

**Part 1** (no actions required)

The dataset is composed of a set of sentences. Each sentence is a list of tuples of a word and a tag, as provided by human annotators.
You should split the data to train and test sets in the following way:


In [1]:
import numpy as np
import operator
import nltk
from nltk.corpus import treebank 
nltk.download('treebank')
print(f"Number of sentences: {len(treebank.tagged_sents())}")

train_data = treebank.tagged_sents()[:3000] 
test_data = treebank.tagged_sents()[3000:] 
print(train_data[0])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
Number of sentences: 3914
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


**Part 2**

Write a class simple_tagger, with methods *train* and *evaluate*. The method *train* receives the data as a list of treebank sentences, as presented above, and use it for training the tagger. In this case, it should learn a simple map of words to tags, defined as the most frequent tag for every word (in case there is more than one tag, select one randomly). The map should be stored as a class member for evaluation.

The method evaluate receives the data as a list of treebanl sentences, as presented above, and use it to evaluate the tagger performance. Specifically, it should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the map (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set. The function should return the two numbers: word level accuracy and sentence level accuracy.


In [0]:
from collections import Counter, defaultdict

class simple_tagger:
    def __init__(self):
        self.mft = None
        self.tags_dict = None

    def train(self, data):

        words = defaultdict(lambda: Counter())
        tags = Counter()

        # counter all pairs
        counter = Counter()
        for sen in data:
          for word, tag in sen:
            words[word][tag] += 1
          tags.update([pair[1] for pair in sen])
          counter.update(Counter(sen))

        # most frequent tag/word
        self.mft = tags.most_common(1)[0][0]

        self.tags_dict = dict()
        for word, counter in words.items():
          self.tags_dict[word] = counter.most_common(1)[0][0]

    def evaluate(self, data):
        total_words = 0
        success_words = 0
        success_sentences = 0

        # iterate over all sentences
        for sen in data:
            # count the total number of words
            total_words += len(sen)

            # count match words in sentence
            match_count = sum([(self.tags_dict.get(word, self.mft) == tag) for word, tag in sen])

            # count match words
            success_words += match_count

            # count match sentences
            success_sentences += (match_count == len(sen))

        sentences_stats = success_sentences / len(data)
        words_stats = success_words / total_words
        return words_stats, sentences_stats

In [0]:
st = simple_tagger()
st.train(train_data)
# print(st.tags_dict)
# print(st.mft)

simple_wla, simple_sla = st.evaluate(test_data)

**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence, using the Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. I implemented the Viterbi algorithm for you in the next block, so you can should either plug it into your code or write your own Viterbi version.

Additional guidance:
1. The matrix B represents the probabilities of seeing a word within each POS label.
Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way, every column in B represents the word that it’s id matches the index of the column.
2. During evaluation, you should first convert each word into it’s index and create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see OOV word, and decode every segment individually by Viterbi.


In [0]:
# Viterbi
def viterbi (word_list, A, B, Pi):

    # initialization
    T = len(word_list)
    N = A.shape[0] # number of tags

    delta_table = np.zeros((N, T)) # initialise delta table
    psi = np.zeros((N, T))  # initialise the best path table

    delta_table[:,0] = B[:, word_list[0]] * Pi

    for t in range(1, T):
        for s in range (0, N):
            trans_p = delta_table[:, t-1] * A[:, s]
            psi[s, t], delta_table[s, t] = max(enumerate(trans_p), key=operator.itemgetter(1))
            delta_table[s, t] = delta_table[s, t] * B[s, word_list[t]]

    # Back tracking
    seq = np.zeros(T);
    seq[T-1] =  delta_table[:, T-1].argmax()
    for t in range(T-1, 0, -1):
      #print(seq[t])
      seq[t-1] = psi[int(seq[t]),t]

    return seq

# A simple example to run the algorithm:

# A = np.array([[0.3, 0.7], [0.2, 0.8]])
# B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
# Pi = np.array([0.4, 0.6])
# print(viterbi([3, 3, 3, 3], A, B, Pi))

In [0]:
from collections import Counter, defaultdict
from random import randint

class hmm_tagger:

    def __init__(self):
        self.tags = None
        self.words = None
        self.A = None
        self.B = None
        self.Pi = None

    def train(self, data):

        # tmp - calculate words/tags dictionaries
        words_set = set()
        tags_set = set()
        # tmp - calculate self.A
        tags_next_counter = defaultdict(lambda: Counter())
        # tmp - calculate self.B
        tag_words_counter = defaultdict(lambda: Counter())
        # tmp - calculate self.Pi
        first_tags_counter = Counter()

        for sen in data:
            prev_tag = None
            first_tags_counter[sen[0][1]] += 1
            for word, tag in sen:
                # dictionaries
                words_set.add(word)
                tags_set.add(tag)

                tag_words_counter[tag][word] += 1

                if prev_tag is not None:
                    tags_next_counter[prev_tag][tag] += 1
                prev_tag = tag

        # Calculate words/tags dictionaries
        self.tags = {k: i for i, k in enumerate(tags_set)}
        self.words = {k: i for i, k in enumerate(words_set)}

        # Calculate A
        self.A = np.zeros((len(self.tags), len(self.tags)))
        for tag, next_tags in tags_next_counter.items():
            tag_total = sum(next_tags.values())
            for next_tag in next_tags:
                self.A[self.tags[tag]][self.tags[next_tag]] = next_tags[next_tag] / tag_total

        # Calculate B
        self.B = np.zeros((len(self.tags), len(self.words)))
        for tag, words in tag_words_counter.items():
            tag_total = sum(words.values())
            for word in words:
                self.B[self.tags[tag]][self.words[word]] = words[word] / tag_total

        # Calculate Pi
        self.Pi = np.zeros(len(self.tags))
        total_tags = sum(first_tags_counter.values())
        for tag, count in first_tags_counter.items():
            self.Pi[self.tags[tag]] = count / total_tags

    def evaluate(self, data):
        total_words = 0
        success_words = 0
        sentences_stats = 0
        for sen in data:
            total_words += len(sen)

            match_count = 0
            words_list = []
            tags_list = []
            results_list = []
            for word, tag in sen:
                if word not in self.words:
                    if words_list:
                        results_list.extend(viterbi(words_list, self.A, self.B, self.Pi))
                        words_list = []
                    results_list.append(randint(0, len(self.tags) - 1))
                    tags_list.append(self.tags[tag])
                else:
                    words_list.append(self.words[word])
                    tags_list.append(self.tags[tag])

            if words_list:
                results_list.extend(viterbi(words_list, self.A, self.B, self.Pi))
                words_list = []
            
            match_count += sum(t1 == t2 for t1, t2 in zip(tags_list, results_list))

            success_words += match_count
            sentences_stats += (match_count == len(sen))

        sentences_stats = sentences_stats / len(data)
        words_stats = success_words / total_words
        return words_stats, sentences_stats

In [0]:
hmm = hmm_tagger()
hmm.train(train_data)

In [53]:
print(hmm.tags)
print(hmm.words)
print(hmm.A)
print(hmm.B)
print(hmm.Pi)
print(np.sum(hmm.A[12]))
print(np.sum(hmm.B[22]))
print(np.sum(hmm.Pi))


{'VBZ': 0, 'VBN': 1, 'CC': 2, 'VBG': 3, '-RRB-': 4, 'RB': 5, '-NONE-': 6, '``': 7, '-LRB-': 8, 'EX': 9, '.': 10, 'FW': 11, 'RBR': 12, 'TO': 13, 'IN': 14, 'JJS': 15, 'WP$': 16, 'CD': 17, "''": 18, 'NNS': 19, 'VB': 20, 'MD': 21, ':': 22, 'SYM': 23, 'JJ': 24, 'NNP': 25, 'PRP$': 26, 'RBS': 27, 'WRB': 28, 'UH': 29, ',': 30, 'WP': 31, 'DT': 32, '$': 33, 'PRP': 34, 'WDT': 35, 'VBP': 36, 'NNPS': 37, 'NN': 38, 'POS': 39, 'LS': 40, 'JJR': 41, 'RP': 42, '#': 43, 'VBD': 44, 'PDT': 45}
{'21': 0, 'Already': 1, 'thinks': 2, 'dean': 3, 'model': 4, '*-11': 5, 'when-issued': 6, 'allegedly': 7, 'X': 8, '2-8': 9, 'replicating': 10, 'Dividend': 11, 'fee': 12, 'Champagnes': 13, 'five-point': 14, 'collective': 15, 'label': 16, 'Continuing': 17, 'traders': 18, 'crystal-lattice': 19, 'billion': 20, 'development': 21, 'erroneous': 22, '730': 23, 'negotiating': 24, 'free-enterprise': 25, 'Squier': 26, 'suggest': 27, 'implant': 28, 'Jack': 29, 'moreover': 30, 'markdown': 31, 'legislature': 32, 'chief': 33, '*T*-2

In [54]:
hmm_wla, hmm_sla = hmm.evaluate(test_data)
print ("HMM Tagger: Word level accuarcy is : {:f}; Sentence level accuarcy is {:f}".format(hmm_wla,hmm_sla))

HMM Tagger: Word level accuarcy is : 0.779063; Sentence level accuarcy is 0.089716


**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK, over the test data. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [0]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)



In [0]:
def evaluate_memm(data, tagger):
  success_words = 0
  sentences_stats = 0
  once = 0
  for sentence in data:
    sentence_stats = tagger.evaluate([sentence])
    if sentence_stats == 1:
      sentences_stats += 1

  sentences_stats = sentences_stats / len(data)
  words_stats = tagger.evaluate(test_data)
  return words_stats, sentences_stats

In [28]:
memm_wla, memm_sla = evaluate_memm(test_data, tnt_pos_tagger)
print(memm_wla,memm_sla)

0.875545003237643 0.09846827133479212


In [29]:
import pandas as pd
table = {'simple': [simple_wla, simple_sla],'hmm': [hmm_wla, hmm_sla],'memm':[memm_wla, memm_sla]}

df = pd.DataFrame(table,index = ['word_acc','sentence_acc']) 

print(df)

                simple       hmm      memm
word_acc      0.874120  0.779150  0.875545
sentence_acc  0.079869  0.092998  0.098468


TODO: Print both, word level and sentence level accuracy for all the three taggers in a table.